In [1]:
import sys
import timeit
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from annoy import AnnoyIndex



# Annoy Fast Search

In [3]:
def process_parquet():
    data_train = pd.read_parquet('cf_train.parquet')
    data_val = pd.read_parquet('cf_validation.parquet')
    data_test = pd.read_parquet('cf_test.parquet')
    
    #df_train = data_train.select('user_id','track_id','count')
    #df_val = data_val.select('user_id','track_id','count')
    #df_test = data_test.select('user_id','track_id','count')
    data_train = data_train.rename(columns={'count':'count_play'}) #otherwise row.count will give you method instead of value
    data_train = data_train[['user_id','track_id','count_play']]
    
    return data_train, data_val, data_test

In [4]:
df_train, df_val, df_test = process_parquet()



df_train['user_id'] = df_train['user_id'].apply(hash)
df_train['track_id'] = df_train['track_id'].apply(hash)
df_val['user_id'] = df_val['user_id'].apply(hash)
df_val['track_id'] = df_val['track_id'].apply(hash)
df_test['user_id'] = df_test['user_id'].apply(hash)
df_test['track_id'] = df_test['track_id'].apply(hash)

C:\Users\Jonathan\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [5]:
#print(df_train)
# build tree using ANN
tic = timeit.default_timer()

dim = 3
t = AnnoyIndex(dim, 'euclidean')

for index, row in df_train.iterrows():
    #print(row.count)
    vector = [row.user_id, row.track_id, row.count_play]
    #print(vector)
    t.add_item(index, vector)

t.build(3)

toc = timeit.default_timer()
build_time = toc-tic

print('Time to build tree: {} seconds'.format(build_time))

C:\Users\Jonathan\Anaconda3\lib\site-packages\pyarrow\pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


Time to build tree: 40.4569195 seconds


In [33]:
idx = 100
k = 500 # number of neighbors

tic = timeit.default_timer()
nearest_neighbors = t.get_nns_by_item(idx, k)
toc = timeit.default_timer()

search_time = toc-tic

print('Time to get {}-nearest neighbors for item {} in: {} seconds'.format(k,idx,search_time))

Time to get 500-nearest neighbors for item 100 in: 0.000515899999982139 seconds


In [34]:
print('The {}-nearest neighbors for item {} are in indices: {}'.format(k,idx,nearest_neighbors))
print('\nThese correspond to users with track_id/play count:')
df_train.iloc[nearest_neighbors]

The 500-nearest neighbors for item 100 are in indices: [100, 201578, 477219, 468141, 367839, 139412, 437827, 63450, 119924, 218447, 327752, 383629, 36139, 125037, 438509, 305360, 118620, 193008, 48652, 262520, 277228, 190114, 305749, 9684, 94177, 209302, 232387, 330868, 18928, 487528, 385656, 157742, 267678, 473983, 413623, 46857, 36137, 343752, 387730, 155012, 491284, 471614, 362723, 307247, 241717, 9135, 222377, 322324, 336782, 328577, 366034, 126483, 440043, 178976, 341322, 397909, 291203, 10401, 320606, 99436, 446730, 295983, 271014, 419957, 343015, 332359, 266063, 206385, 56300, 63507, 450964, 254904, 157205, 411226, 267511, 263221, 111158, 346474, 289406, 446896, 38571, 102088, 20113, 354731, 16933, 65140, 279686, 85126, 9956, 16826, 69128, 167454, 72793, 12865, 334674, 400044, 80222, 384768, 327029, 455939, 485303, 297822, 196945, 321328, 396458, 366033, 350577, 57232, 138841, 226387, 365670, 176509, 381594, 302975, 406626, 241531, 38612, 172888, 187831, 394356, 227777, 95953, 1

,user_id,track_id,count_play
100,7136566080545880984,2887313970437931179,1
201578,7148227064974373113,2865135697155157573,1
477219,7125515783895567047,2912573526146869598,4
468141,7124301859171593566,2858493847568314791,7
367839,7160689515887929237,2910224100610506036,12
...,...,...,...
490482,7266936298570642344,3197813985264415667,1
459167,7472061914667602070,2920841817089190485,1
189340,7139041730276222834,2549169297640746802,1
360232,6892618954263281437,2652793360904765384,1


# Brute Force Search

In [6]:
# we want to measure the distance of the vectors from the other indices to the vector of the source index
# define the source vector to be at index = idx = 100

# build
tic = timeit.default_timer()

idx = 100
source = df_train.loc[idx].to_numpy()
distances = []

for index, row in df_train.iterrows():
    distance = euclidean_distances(source.reshape(1,-1), row.to_numpy().reshape(1,-1))[0]
    distances.append(distance)
    
toc = timeit.default_timer()
build_time = toc - tic

In [7]:
print('Time to build distances array: {} seconds'.format(build_time))

Time to build distances array: 80.7128257 seconds


In [8]:
# search
tic = timeit.default_timer()
nearest_neighbors = np.argmin(distances)

toc = timeit.default_timer()
search_time = toc-tic

In [9]:
print('Time to get nearest neighbors for item {} in: {} seconds'.format(idx,search_time))

Time to get nearest neighbors for item 100 in: 0.18988110000000802 seconds


In [10]:
print('The nearest neighbor for item {} is in index: {}'.format(idx,nearest_neighbors))
print('\nThis correspond to user with track_id/play count:')
df_train.iloc[nearest_neighbors]

The nearest neighbor for item 100 is in index: 100

This correspond to user with track_id/play count:


user_id       7096748636088034474
track_id     -7712253977634981864
count_play                      1
Name: 100, dtype: int64

In [11]:
print('Approximate nearest neighbors build time was approximately {}x faster'.format(80.7128257/40.4569195))
print('Approximate nearest neighbors search performed approximately {}x faster'.format(0.18988110000000802/0.000515899999982139))

Approximate nearest neighbors build time was approximately 1.9950314234873963x faster
Approximate nearest neighbors search performed approximately 368.0579569811628x faster
